In [124]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import datetime
import pickle
import gzip
import matplotlib.pyplot as plt

The aim of this notebook is to 
identify treat VS control (video that talks about the release event VS not ) —> from the title /metatda of video
classify - treat VS control
compare graphs – analyse channels‘growth for each relase ( 1 graph/ release )
for each event : analyse of all channels growth == what is the evolution of the nb of views and comments 


1. classify channels in the treat control group,
2. causal observation (like lab 5) and outcome nb of subscribers one month after the event 

In [125]:
tech_video_metadata = "/Users/salma/Desktop/ADA/project/datasets/tech_review_videos.tsv.gz"
video_comments ="/Users/salma/Desktop/ADA/project/datasets/num_comments.tsv.gz"
tech_channels_metadata = "/Users/salma/Desktop/ADA/project/datasets/tech_review_channels.tsv.gz"

In [126]:
ITEMS = ['iphone x', 'iphone 7', 'iphone 6', 'galaxy s9', 'galaxy note 9', 'galaxy fold']
RELEASE_DATES = {
    'iphone x': '2017-11-03'
#    'iphone 7': '2016-09-16',
#    'iphone 6': '2014-09-19',
#    'galaxy s9': '2018-03-16',
#    'galaxy note 9': '2018-08-24',
#    'galaxy fold': '2019-02-01'
}


In [127]:
#with gzip.open(tech_video_metadata, "rb") as f:
 #   df_video = pickle.load(f, encoding='bytes')
#display(df_video)

In [128]:
df_video = pd.read_table(tech_video_metadata)
df_channel = pd.read_table(tech_channels_metadata)
df_comments = pd.read_table(video_comments)

In [129]:
df_video


,channel_id,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count,num_comms,clean_title,clean_tags
0,UCzWQYUVCpZqtN93H8RR44Qw,87.0,ExOXF1x3N1g,292,7386.0,"current events,Science,biofuel,algae,power,Bio...",We Can Power The World With Algae!,2016-09-29,235974.0,700.0,"['power', 'world', 'algae']","['current', 'event', 'science', 'biofuel', 'al..."
1,UCzWQYUVCpZqtN93H8RR44Qw,36.0,_RhHuW2gglw,193,1933.0,"Energy (Dimension),testtube,fertilizer,human w...",How Poop Will Power The World,2013-09-15,92531.0,465.0,"['poop', 'power', 'world']","['energy', 'dimension', 'testtube', 'fertilize..."
2,UCzWQYUVCpZqtN93H8RR44Qw,34.0,g513EUHqkfw,191,2556.0,"social,networking,first impressions,judge,meet...",How First Impressions Work,2012-12-10,164020.0,248.0,"['first', 'impression', 'work']","['social', 'networking', 'first', 'impression'..."
3,UCzVy1DW9NFp4c910hMd_jIw,1.0,fRY3dpFBF14,217,72.0,"Sony,8K,Sony 8K,TV,2019 Sony 8K,8K Television,...",Sony 8K Z9G Master Series Smart TV,2019-08-23,2585.0,12.0,"['sony', 'master', 'series', 'smart', 'tv']","['sony', 'sony', 'tv', 'sony', 'television', '..."
4,UCzVy1DW9NFp4c910hMd_jIw,3.0,VenwYvLRa-I,437,107.0,"Bowers and Wilkins,B&W,Bowers,Wilkins,603,606,...",Bowers & Wilkins 600 Series Speaker Comparison,2019-08-13,5974.0,19.0,"['bower', 'wilkins', 'series', 'speaker', 'com...","['bower', 'wilkins', 'bower', 'wilkins', 'cent..."
...,...,...,...,...,...,...,...,...,...,...,...,...
178250,UCrwT2LIOntbglCR2o0iOXVw,20.0,WjcXvzzRB0M,196,486.0,"surface book,best laptop for engineering,elect...",The Best Engineering Laptop!Mac or PC?,2018-07-08,42628.0,79.0,"['best', 'engineering', 'laptop', 'mac', 'pc']","['surface', 'book', 'best', 'laptop', 'enginee..."
178251,UCrwPkTBuhZJDgiHkWzY7COw,35.0,UySYfgQaIlo,200,31.0,"how to install windows 10 on android phone,win...",How to Install Windows 10 on Any Android Phone...,2018-10-20,2723.0,7.0,"['install', 'window', 'android', 'phone']","['install', 'window', 'android', 'phone', 'win..."
178252,UCrwPkTBuhZJDgiHkWzY7COw,0.0,lqpuuKkh-lA,226,11.0,"Best Video player for Android,video player,vid...",Best Video Player For Android Phone - Must Try!,2018-06-19,897.0,0.0,"['best', 'video', 'player', 'android', 'phone'...","['best', 'video', 'player', 'android', 'video'..."
178253,UCrwMvZOgSvS2srsIjY-iIGg,0.0,7uFD7TYwlW4,182,7.0,NaN,I PHONE 11MAX PRO REVIEW,2019-09-21,54.0,0.0,"['phone', 'max', 'pro', 'review']",[]


In [130]:
df_channel

,join_date,channel_id,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights,count_tech_videos
0,2010-12-21,UCsTcErHg8oDvUnTzoqsYeNw,Unbox Therapy,15300000,1672,206.0,2.0870,406.0
1,2015-10-19,UCOhHO2ICt0ti9KAh-QHvttQ,Technical Guruji,13979890,2539,240.0,2.0870,209.0
2,2012-05-18,UCe_vXdMrHHseZ_esYUskSBw,CrazyRussianHacker,11200000,1411,444.0,2.0870,105.0
3,2005-06-22,UCE_M8A5yxnLfW0KghEeajjw,Apple,9970000,291,489.0,2.0870,64.0
4,2008-03-21,UCBJycsmduvYEL83R_U4JriQ,Marques Brownlee,9400000,1148,530.0,2.0870,370.0
...,...,...,...,...,...,...,...,...
1188,2012-06-06,UCgMHqEDFll6X962CgXPzh4w,FocusCamera TV,10000,278,940963.0,40.6530,42.0
1189,2018-02-25,UCJjacdkHH7Zxw8ouKQrirUg,The Best Coders,10200,28,941074.0,42.4845,2.0
1190,2016-07-17,UCviAcL8AlEoGqLDWx7t_iPw,Mobile Tag,10300,20,952387.0,53.1435,6.0
1191,2011-02-20,UCncAHfjOv1B_U8esTmLYukQ,Curtis Paradis,10300,97,963290.0,53.1435,11.0


#### treat_i is equal to 1, this means that the video talks about the event. Otherwise, it does not.

In [131]:

df_video_cop = df_video.copy()
df_channel_cop = df_channel.copy()

df_merged = df_video_cop.merge(df_channel_cop,on=['channel_id'],how ='left')

df_merged["treat"]=0




In [132]:
df_merged["tags"] = df_merged["tags"].str.lower()
df_merged["title"]= df_merged["title"].str.lower()
df_merged

,channel_id,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count,num_comms,clean_title,clean_tags,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights,count_tech_videos,treat
0,UCzWQYUVCpZqtN93H8RR44Qw,87.0,ExOXF1x3N1g,292,7386.0,"current events,science,biofuel,algae,power,bio...",we can power the world with algae!,2016-09-29,235974.0,700.0,"['power', 'world', 'algae']","['current', 'event', 'science', 'biofuel', 'al...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,UCzWQYUVCpZqtN93H8RR44Qw,36.0,_RhHuW2gglw,193,1933.0,"energy (dimension),testtube,fertilizer,human w...",how poop will power the world,2013-09-15,92531.0,465.0,"['poop', 'power', 'world']","['energy', 'dimension', 'testtube', 'fertilize...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,UCzWQYUVCpZqtN93H8RR44Qw,34.0,g513EUHqkfw,191,2556.0,"social,networking,first impressions,judge,meet...",how first impressions work,2012-12-10,164020.0,248.0,"['first', 'impression', 'work']","['social', 'networking', 'first', 'impression'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,UCzVy1DW9NFp4c910hMd_jIw,1.0,fRY3dpFBF14,217,72.0,"sony,8k,sony 8k,tv,2019 sony 8k,8k television,...",sony 8k z9g master series smart tv,2019-08-23,2585.0,12.0,"['sony', 'master', 'series', 'smart', 'tv']","['sony', 'sony', 'tv', 'sony', 'television', '...",2010-10-11,Audio Advice,19500.0,199.0,416394.0,8.1235,25.0,0
4,UCzVy1DW9NFp4c910hMd_jIw,3.0,VenwYvLRa-I,437,107.0,"bowers and wilkins,b&w,bowers,wilkins,603,606,...",bowers & wilkins 600 series speaker comparison,2019-08-13,5974.0,19.0,"['bower', 'wilkins', 'series', 'speaker', 'com...","['bower', 'wilkins', 'bower', 'wilkins', 'cent...",2010-10-11,Audio Advice,19500.0,199.0,416394.0,8.1235,25.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178250,UCrwT2LIOntbglCR2o0iOXVw,20.0,WjcXvzzRB0M,196,486.0,"surface book,best laptop for engineering,elect...",the best engineering laptop!mac or pc?,2018-07-08,42628.0,79.0,"['best', 'engineering', 'laptop', 'mac', 'pc']","['surface', 'book', 'best', 'laptop', 'enginee...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
178251,UCrwPkTBuhZJDgiHkWzY7COw,35.0,UySYfgQaIlo,200,31.0,"how to install windows 10 on android phone,win...",how to install windows 10 on any android phone...,2018-10-20,2723.0,7.0,"['install', 'window', 'android', 'phone']","['install', 'window', 'android', 'phone', 'win...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
178252,UCrwPkTBuhZJDgiHkWzY7COw,0.0,lqpuuKkh-lA,226,11.0,"best video player for android,video player,vid...",best video player for android phone - must try!,2018-06-19,897.0,0.0,"['best', 'video', 'player', 'android', 'phone'...","['best', 'video', 'player', 'android', 'video'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
178253,UCrwMvZOgSvS2srsIjY-iIGg,0.0,7uFD7TYwlW4,182,7.0,NaN,i phone 11max pro review,2019-09-21,54.0,0.0,"['phone', 'max', 'pro', 'review']",[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [133]:
df_merged = df_merged.loc[(df_merged["upload_date"] > '2017-11-03')& (df_merged["upload_date"] <'2017-12-03')]



,channel_id,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count,num_comms,clean_title,clean_tags,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights,count_tech_videos,treat
33,UCzV07t6mx27AcEPmpAUJYXw,3.0,2g3ykHWEOsI,338,28.0,"rhino slider,rhino slider review,youtube video...",the best motion camera slider for youtube videos!,2017-11-28,805.0,3.0,"['best', 'motion', 'camera', 'slider', 'youtub...","['rhino', 'slider', 'rhino', 'slider', 'review...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
34,UCzV07t6mx27AcEPmpAUJYXw,86.0,HWMm3ERfFCs,324,142.0,"overhead camera,overhead camera mount,overhead...",the best overhead camera rig for youtube videos!,2017-11-26,21197.0,22.0,"['best', 'overhead', 'camera', 'rig', 'youtube...","['overhead', 'camera', 'overhead', 'camera', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
35,UCzV07t6mx27AcEPmpAUJYXw,45.0,6xVfMLhsdRc,585,256.0,"google home mini,google home mini unboxing ind...",google home mini in india - unboxing & setup!,2017-11-19,15527.0,157.0,"['google', 'home', 'mini', 'india', 'unboxing'...","['google', 'home', 'mini', 'google', 'home', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
36,UCzV07t6mx27AcEPmpAUJYXw,9.0,EWfSia7qNqI,139,24.0,"soumendra jena,sj,sj channel,soamjena,apple ip...",how to remove the top notch on apple iphone x!,2017-11-11,1247.0,7.0,"['remove', 'top', 'notch', 'apple', 'iphone']","['soumendra', 'jena', 'sj', 'sj', 'channel', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
37,UCzV07t6mx27AcEPmpAUJYXw,3.0,lFD3YQgZC-o,478,63.0,"iphone x review,iphone 10 review,iphone x full...",apple iphone x - the detailed review!,2017-11-11,1300.0,63.0,"['apple', 'iphone', 'detailed', 'review']","['iphone', 'review', 'iphone', 'review', 'ipho...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177760,UCsCex8aMDs2b39TS_49Mpcg,17.0,MZk2ctF6tpI,714,551.0,"lg v30 review,lg v30 vs note 8,lg v30 unboxing...",the best review of the most under rated phone ...,2017-12-02,18241.0,142.0,"['best', 'review', 'rated', 'phone']","['lg', 'review', 'lg', 'note', 'lg', 'unboxing...",2013-10-23,Jonathan Casey,74300.0,560.0,192469.0,5.741,165.0,0
177898,UCs7qwxT0HL2fLTtVrt4i45w,4.0,EgQUk0DBs_c,286,66.0,"how i plan out youtube videos,planning youtube...",how i plan out youtube videos,2017-11-24,1766.0,19.0,"['plan', 'youtube', 'video']","['plan', 'youtube', 'video', 'planning', 'yout...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
177966,UCs6KfncB4OV6Vug4o_bzijg,25.0,Y3HwTwYliwY,484,372.0,"protonvpn free,protonvpn review,protonvpn,prot...",protonvpn full complete review! the new best vpn?,2017-11-18,28321.0,98.0,"['protonvpn', 'full', 'complete', 'review', 'n...","['protonvpn', 'free', 'protonvpn', 'review', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
177978,UCs11Yv5OnpY_b3Bff5ZJ9kw,564.0,PcgcD8HQb0M,383,19344.0,"iphone,iphonex,unboxing,apple,jordyn,jordyn jones",iphone x first impression/unboxing,2017-11-04,693417.0,0.0,"['iphone', 'first', 'impression', 'unboxing']","['iphone', 'iphonex', 'unboxing', 'apple', 'jo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [152]:
display(df_merged)

,channel_id,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count,num_comms,clean_title,clean_tags,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights,count_tech_videos,treat
33,UCzV07t6mx27AcEPmpAUJYXw,3.0,2g3ykHWEOsI,338,28.0,"rhino slider,rhino slider review,youtube video...",the best motion camera slider for youtube videos!,2017-11-28,805.0,3.0,"['best', 'motion', 'camera', 'slider', 'youtub...","['rhino', 'slider', 'rhino', 'slider', 'review...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
34,UCzV07t6mx27AcEPmpAUJYXw,86.0,HWMm3ERfFCs,324,142.0,"overhead camera,overhead camera mount,overhead...",the best overhead camera rig for youtube videos!,2017-11-26,21197.0,22.0,"['best', 'overhead', 'camera', 'rig', 'youtube...","['overhead', 'camera', 'overhead', 'camera', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
35,UCzV07t6mx27AcEPmpAUJYXw,45.0,6xVfMLhsdRc,585,256.0,"google home mini,google home mini unboxing ind...",google home mini in india - unboxing & setup!,2017-11-19,15527.0,157.0,"['google', 'home', 'mini', 'india', 'unboxing'...","['google', 'home', 'mini', 'google', 'home', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
36,UCzV07t6mx27AcEPmpAUJYXw,9.0,EWfSia7qNqI,139,24.0,"soumendra jena,sj,sj channel,soamjena,apple ip...",how to remove the top notch on apple iphone x!,2017-11-11,1247.0,7.0,"['remove', 'top', 'notch', 'apple', 'iphone']","['soumendra', 'jena', 'sj', 'sj', 'channel', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
37,UCzV07t6mx27AcEPmpAUJYXw,3.0,lFD3YQgZC-o,478,63.0,"iphone x review,iphone 10 review,iphone x full...",apple iphone x - the detailed review!,2017-11-11,1300.0,63.0,"['apple', 'iphone', 'detailed', 'review']","['iphone', 'review', 'iphone', 'review', 'ipho...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177760,UCsCex8aMDs2b39TS_49Mpcg,17.0,MZk2ctF6tpI,714,551.0,"lg v30 review,lg v30 vs note 8,lg v30 unboxing...",the best review of the most under rated phone ...,2017-12-02,18241.0,142.0,"['best', 'review', 'rated', 'phone']","['lg', 'review', 'lg', 'note', 'lg', 'unboxing...",2013-10-23,Jonathan Casey,74300.0,560.0,192469.0,5.741,165.0,1
177898,UCs7qwxT0HL2fLTtVrt4i45w,4.0,EgQUk0DBs_c,286,66.0,"how i plan out youtube videos,planning youtube...",how i plan out youtube videos,2017-11-24,1766.0,19.0,"['plan', 'youtube', 'video']","['plan', 'youtube', 'video', 'planning', 'yout...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
177966,UCs6KfncB4OV6Vug4o_bzijg,25.0,Y3HwTwYliwY,484,372.0,"protonvpn free,protonvpn review,protonvpn,prot...",protonvpn full complete review! the new best vpn?,2017-11-18,28321.0,98.0,"['protonvpn', 'full', 'complete', 'review', 'n...","['protonvpn', 'free', 'protonvpn', 'review', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
177978,UCs11Yv5OnpY_b3Bff5ZJ9kw,564.0,PcgcD8HQb0M,383,19344.0,"iphone,iphonex,unboxing,apple,jordyn,jordyn jones",iphone x first impression/unboxing,2017-11-04,693417.0,0.0,"['iphone', 'first', 'impression', 'unboxing']","['iphone', 'iphonex', 'unboxing', 'apple', 'jo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [149]:
df_merged.loc[df_merged["title"].str.contains("iphone x"),"treat"] = '1' 


In [150]:
#df_merged.fillna({'tags': 'XXX'})
#df_merged
df_merged.loc[df_merged["tags"].str.contains("iphone x",na=False),"treat"] = '1' 
#ex of disply id of video that talks about iphone x : lFD3YQgZC-o

In [154]:
display(df_merged.loc[df_merged['treat'] == '1'])

,channel_id,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count,num_comms,clean_title,clean_tags,join_date,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights,count_tech_videos,treat
36,UCzV07t6mx27AcEPmpAUJYXw,9.0,EWfSia7qNqI,139,24.0,"soumendra jena,sj,sj channel,soamjena,apple ip...",how to remove the top notch on apple iphone x!,2017-11-11,1247.0,7.0,"['remove', 'top', 'notch', 'apple', 'iphone']","['soumendra', 'jena', 'sj', 'sj', 'channel', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
37,UCzV07t6mx27AcEPmpAUJYXw,3.0,lFD3YQgZC-o,478,63.0,"iphone x review,iphone 10 review,iphone x full...",apple iphone x - the detailed review!,2017-11-11,1300.0,63.0,"['apple', 'iphone', 'detailed', 'review']","['iphone', 'review', 'iphone', 'review', 'ipho...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
38,UCzV07t6mx27AcEPmpAUJYXw,2.0,O4AoiYxgEek,139,13.0,"soumendra jena,sj,sj channel,soamjena,iphone x...",camera comparison : apple iphone x vs samsung s8+,2017-11-09,338.0,7.0,"['camera', 'comparison', 'apple', 'iphone', 's...","['soumendra', 'jena', 'sj', 'sj', 'channel', '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
271,UCzLaQ6eeTVuAltzTrN7fzyg,49.0,j8N0_hpIbOg,371,164.0,"youtube tech guy,the android guy,cellphone,sma...",samsung gear fit 2 pro review - youtube tech guy,2017-12-01,25708.0,16.0,"['samsung', 'gear', 'fit', 'pro', 'review', 'y...","['youtube', 'tech', 'guy', 'android', 'guy', '...",2012-02-16,The YouTube Tech Guy...,314000.0,3205.0,56986.0,3.532,1040.0,1
272,UCzLaQ6eeTVuAltzTrN7fzyg,134.0,uD3jCBz-Wdk,595,191.0,"youtube tech guy,the android guy,cellphone,sma...",pixel 2 xl review - one of the most disappoint...,2017-11-30,5828.0,103.0,"['pixel', 'xl', 'review', 'one', 'disappointin...","['youtube', 'tech', 'guy', 'android', 'guy', '...",2012-02-16,The YouTube Tech Guy...,314000.0,3205.0,56986.0,3.532,1040.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176040,UCsTcErHg8oDvUnTzoqsYeNw,30050.0,2fGXDFiFBhg,503,307915.0,"iphone x,iphone x review,iphone,review,apple i...",don't buy the iphone x,2017-11-26,10912434.0,58602.0,"['buy', 'iphone']","['iphone', 'iphone', 'review', 'iphone', 'revi...",2010-12-21,Unbox Therapy,15300000.0,1672.0,206.0,2.087,406.0,1
176041,UCsTcErHg8oDvUnTzoqsYeNw,2064.0,Rh_iWXRxIY4,365,39646.0,"razer phone,razer,phone,speaker,razer phone sp...","how loud is the razer phone? (vs iphone x, pix...",2017-11-18,1293524.0,2945.0,"['loud', 'razer', 'phone', 'iphone', 'pixel', ...","['razer', 'phone', 'razer', 'phone', 'speaker'...",2010-12-21,Unbox Therapy,15300000.0,1672.0,206.0,2.087,406.0,1
176042,UCsTcErHg8oDvUnTzoqsYeNw,2555.0,0KcvQpRdCzQ,402,83347.0,"oneplus 5t,oneplus 5t unboxing,oneplus 5t revi...",oneplus 5t unboxing - is this the one?,2017-11-16,4779536.0,7205.0,"['oneplus', 'unboxing', 'one']","['oneplus', 'oneplus', 'unboxing', 'oneplus', ...",2010-12-21,Unbox Therapy,15300000.0,1672.0,206.0,2.087,406.0,1
177760,UCsCex8aMDs2b39TS_49Mpcg,17.0,MZk2ctF6tpI,714,551.0,"lg v30 review,lg v30 vs note 8,lg v30 unboxing...",the best review of the most under rated phone ...,2017-12-02,18241.0,142.0,"['best', 'review', 'rated', 'phone']","['lg', 'review', 'lg', 'note', 'lg', 'unboxing...",2013-10-23,Jonathan Casey,74300.0,560.0,192469.0,5.741,165.0,1


In [135]:
#if (df_merged.groupby("channel_id").sum > 0) :
 #   df_merged["treat"] = 1

TypeError: '>' not supported between instances of 'method' and 'int'